# 用库实现线性回归

In [3]:
import torch
import numpy as np

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [4]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [5]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.0499, -1.1531],
        [-0.2964, -1.2811],
        [-0.4684, -1.1136],
        [ 0.8159,  1.2036],
        [-0.8677, -0.2604],
        [-0.0330, -0.4211],
        [ 1.6479, -0.2570],
        [-1.5399, -2.0537],
        [ 0.8645,  0.0306],
        [ 0.6216,  1.2544]]) tensor([8.2248, 7.9612, 7.0527, 1.7468, 3.3548, 5.5750, 8.3688, 8.1009, 5.8099,
        1.1765])


In [10]:
import torch.nn as nn

class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    def forward(self,x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [12]:
# 也可以直接叠床架屋的形式来构造网络

# Sequential是一个容器，用于把所有的层串联起来/包起来
net = nn.Sequential()
net.add_module("linear", nn.Linear(num_inputs,1))

print(net)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [13]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.5777, 0.1491]], requires_grad=True)
Parameter containing:
tensor([-0.4032], requires_grad=True)


In [16]:
from torch.nn import init

init.normal_(net[0].weight, mean = 0, std = 0.01)
init.constant_(net[0].bias, val = 0)

Parameter containing:
tensor([0.], requires_grad=True)

In [17]:
loss = nn.MSELoss()

In [18]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [19]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000193
epoch 2, loss: 0.000129
epoch 3, loss: 0.000061


In [20]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9998, -3.3993]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2000], requires_grad=True)
